In [1]:
import os
import datetime as dt
import pandas as pd
import numpy as np
import time

main_dir = os.getcwd()
os.chdir(main_dir + '\\Leaps Pulling')
cad_dir = main_dir + '\\Historical Queries\\CAD'
usd_dir = main_dir + '\\Historical Queries\\US'

from helpers import *
from yahoo_query import *

os.chdir(main_dir)

In [2]:
def create_dgi(annual_data):
    dgi_data = annual_data[['totalLiab',
                             'totalStockholderEquity',
                             'longTermInvestments',
                             'shortTermInvestments',
                             'netIncome','operatingIncome',
                             'totalRevenue','dividendsPaid',
                             'investments','Underlying']]
    dgi_data['dividendsPaid'] = -dgi_data['dividendsPaid']

    dgi_data['payoutRatio'] = dgi_data.dividendsPaid/dgi_data.netIncome
    dgi_data['debtEquityRatio'] = dgi_data.totalLiab/dgi_data.totalStockholderEquity
    dgi_data['netMargin'] = dgi_data.netIncome/dgi_data.totalRevenue
    dgi_data['roic'] = dgi_data.operatingIncome/(dgi_data.longTermInvestments + dgi_data.shortTermInvestments)

    dgi_data = dgi_data[['Underlying','payoutRatio','debtEquityRatio','netMargin','roic','dividendsPaid']]
    return dgi_data

# DGI Portion

def dgi_scores(annual_df,keystats, min_dgi_score):
    dgi_df = create_dgi(annual_df).fillna(0).replace(-np.inf,0).replace(np.inf,0)

    dgi_df_scores = pd.DataFrame(columns = ['divGrowth','payoutChange','payoutRatio',
                                             'debtEquityRatio','netMargin','roic'],
                                  index = dgi_df.Underlying.drop_duplicates().tolist()).fillna(0)

    for ticker in dgi_df.Underlying.drop_duplicates():
        curr_dgi = dgi_df[dgi_df.Underlying == ticker].sort_index()
        if len(curr_dgi) - 1:
            if sum(curr_dgi['dividendsPaid'].pct_change() >= 0.02) == len(curr_dgi) -1:
                dgi_df_scores.loc[ticker, 'divGrowth'] = 1
            if sum(curr_dgi.payoutRatio.pct_change() <= 1) == len(curr_dgi) - 1:
                dgi_df_scores.loc[ticker, 'payoutChange'] = 1
            if curr_dgi.payoutRatio.tail(3).mean() <= 0.6:
                dgi_df_scores.loc[ticker, 'payoutRatio'] = 1
            if sum(curr_dgi.debtEquityRatio.tail(3) <= 1.5) == len(curr_dgi) - 1:
                dgi_df_scores.loc[ticker, 'debtEquityRatio'] = 1
            if sum(curr_dgi.netMargin.tail(3) >= 0.07) == len(curr_dgi) - 1:
                dgi_df_scores.loc[ticker, 'netMargin'] = 1
            if sum(curr_dgi.roic.tail(3) >= 0.2) == len(curr_dgi) - 1:
                dgi_df_scores.loc[ticker, 'roic'] = 1

    dgi_df_scores['score'] = dgi_df_scores.sum(axis = 1)
    dgi_df_scores = dgi_df_scores[dgi_df_scores.score >= min_dgi_score].join(keystats[['currentPrice','returnOnAssets',
                                                                                       'returnOnEquity','sector']])

    dgi_df_scores = dgi_df_scores.sort_values('returnOnAssets', ascending = False)
    dgi_df_scores['roaRank'] = range(1,len(dgi_df_scores) + 1)
    dgi_df_scores = dgi_df_scores.sort_values('returnOnEquity', ascending = False)
    dgi_df_scores['roeRank'] = range(1,len(dgi_df_scores) + 1)
    dgi_df_scores['roRank'] = dgi_df_scores.roaRank + dgi_df_scores.roeRank
    dgi_df_scores = dgi_df_scores.sort_values('roRank')
    return dgi_df_scores[['currentPrice','score','roRank','sector']]

# filterd_dgi = dgi_df[(dgi_df['debtEquityAvg'] <= 1.5) &
#        (dgi_df['divGrowth'] >= 0.03) &
#        (dgi_df['payoutChange'] <= 0) &
#        (dgi_df['payoutAvg'] <= 0.6) & (dgi_df['payoutAvg'] >= 0) &
#        (dgi_df['netMarginAvg'] >= 0.07) &
#        (dgi_df['roicAvg'] >= 0.2)].sort_values('divGrowth', ascending = False)



# LEAPs Portion

def leaps_scores(data_df, keystats, min_score):
    try:
        df = data_df[['Underlying', 'earnings', 'netIncome','totalRevenue','totalStockholderEquity']]
    except:
        df = data_df[['Underlying', 'epsActual', 'netIncome','totalRevenue','totalStockholderEquity']]
    ks = keystats[['52WeekChange','currentPrice','sector','forwardPE','forwardEps','pegRatio']]
    ks['52WeekChange'] = pd.to_numeric(ks['52WeekChange'].replace('Infinity',0))

    df['profitMargin'] = df.netIncome/df.totalRevenue
    df['roe'] = df.netIncome/df.totalStockholderEquity

    # us_quarterly[['Underlying', 'retainedEarnings', 'netIncome','totalRevenue','totalStockholderEquity']]

    leap_scores = pd.DataFrame(columns = ['earningsGrowth','profitMargins','profitMarginChange',
                                          'roe','roeChange'],
                               index = df.Underlying.drop_duplicates().tolist()).fillna(0)

    for ticker in df.Underlying.drop_duplicates():
        curr_data = df[df.Underlying == ticker].sort_index()
        curr_len = len(curr_data) - 1
        if curr_len > 1:
            try:
                if sum(curr_data.earnings.pct_change() >= 0.05) == curr_len:
                    leap_scores.loc[ticker, 'earningsGrowth'] = 1
            except:
                if sum(curr_data.epsActual.pct_change() > 0) == curr_len:
                    leap_scores.loc[ticker, 'earningsGrowth'] = 1
            if sum(curr_data.profitMargin.tail(3) >= 0.1) == curr_len:
                leap_scores.loc[ticker, 'profitMargins'] = 1
            if sum(curr_data.profitMargin.pct_change() >= 0) == curr_len:
                leap_scores.loc[ticker, 'profitMarginChange'] = 1
            if sum(curr_data.roe.tail(3) >= 0.1) == curr_len:
                leap_scores.loc[ticker, 'roe'] = 1
            if sum(curr_data.roe.pct_change() >= 0) == curr_len:
                leap_scores.loc[ticker, 'roeChange'] = 1

    leap_scores['score'] = leap_scores.sum(axis = 1)
    leap_scores = leap_scores[leap_scores.score >= min_score].join(ks).sort_values('52WeekChange', ascending = False)
    return leap_scores


# DGI Filter

In [ ]:
os.chdir(cad_dir)

file_date = '2018-09-19'
cad_annual = pd.read_csv('cad_annual_{}.csv'.format(file_date), index_col = 0)
cad_keystats = pd.read_csv('cad_keystats_{}.csv'.format(file_date), index_col = 0)
cad_quarterly = pd.read_csv('cad_quarterly_{}.csv'.format(file_date), index_col = 0)

os.chdir(main_dir)

os.chdir(usd_dir)

file_date = '2018-09-19'
us_annual = pd.read_csv('us_annual_{}.csv'.format(file_date), index_col = 0)
us_keystats = pd.read_csv('us_keystats_{}.csv'.format(file_date), index_col = 0)
us_quarterly = pd.read_csv('us_quarterly_{}.csv'.format(file_date), index_col = 0)

os.chdir(main_dir)

In [ ]:
cad_dgi_scores = dgi_scores(cad_annual, cad_keystats, 4)
print("CAD")
cad_dgi_scores.head(20)

In [ ]:
us_dgi_score = dgi_scores(us_annual, us_keystats, 5)
print("USD")
us_dgi_score.head(20)

# LEAPs Filtering

In [ ]:
leaps_annual = leaps_scores(us_annual, us_keystats, 4)#.replace('Infinity', 0)
leaps_quarterly = leaps_scores(us_quarterly, us_keystats, 4)#.replace('Infinity', 0)


In [ ]:
leaps_annual[(leaps_annual.profitMargins > 0) &
             (leaps_annual.earningsGrowth > 0) &
             (leaps_annual.roe > 0) &
             (leaps_annual.profitMarginChange > 0) &
             (pd.to_numeric(leaps_annual['52WeekChange']) > 0.1)]

In [ ]:
leaps_quarterly

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
os.chdir(main_dir + '\\Single Name Pulls')
datenow = dt.datetime.today().strftime("%Y-%m-%d")
filename = 'single_names {}.xlsx'.format(datenow)

writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Write each dataframe to a different worksheet.
cad_dgi_scores.to_excel(writer, sheet_name='ca_dgi')
us_dgi_score.to_excel(writer, sheet_name='us_dgi')
leaps_annual.to_excel(writer, sheet_name='leaps_a')
leaps_quarterly.to_excel(writer, sheet_name = 'leaps_q')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
os.chdir(main_dir)

In [ ]:
# Weed Names

modules = '%2C'.join(['assetProfile','incomeStatementHistory', 'balanceSheetHistoryQuarterly',
                              'balanceSheetHistory','cashflowStatementHistory', 'cashflowStatementHistoryQuarterly',
                              'defaultKeyStatistics','financialData','incomeStatementHistoryQuarterly',
                              'calendarEvents','secFilings', 'recommendationTrend', 'institutionOwnership',
                              'fundOwnership', 'majorDirectHolders', 'majorHoldersBreakdown',
                              'insiderTransactions', 'insiderHolders', 'netSharePurchaseActivity',
                              'earnings', 'earningsHistory', 'earningsTrend', 'industryTrend', 'indexTrend',
                              'sectorTrend'])


tickers = ['ACAN', 'ACBFF', 'APHQF', 'CANN',
           'CBDS', 'CBIS', 'CNAB', 'TLRY',
           'CGC', 'CRON']

weed_ks = []
weed_classes = []
for ticker in tickers:
    full_info_url = 'https://query1.finance.yahoo.com/v10/finance/quoteSummary/{0}?modules={1}'.format(ticker, modules)
    with urlreq.urlopen(full_info_url) as url:
        data = json.loads(url.read().decode())
    weed_classes.append(data)
    
    keyStats = pd.DataFrame(data['quoteSummary']['result'][0]['defaultKeyStatistics']).loc[['raw']]
    keyStats.index = [ticker]
    weed_ks.append(keyStats)

In [ ]:
weed_ks.drop(weed_ks.isnull().sum().T.sort_values().tail(19).index.tolist(),axis = 1).T

In [3]:
tlry_chain = all_options('TLRY', 50,1, 1)

tlry_puts = tlry_chain[(tlry_chain.Type == 'put') &
                       (tlry_chain.DTE <= 50) &
                       (tlry_chain.Strike <= tlry_chain.Underlying_Price)].reset_index(drop = True)

long_strike = []
short_strike = []
long_expiry = []
short_expiry = []
cost = []
new_price = []


curr_sim = price_sim(tlry_puts, -0.3, -0.1, 3, output = 'All',
                     skew = 'flat', day_format = 'trading', interest_rate = 0.0193, q = 0,
                     prem_price_use = 'Mid')

for dte in tlry_puts.DTE.drop_duplicates():
    curr_puts = tlry_puts[tlry_puts.DTE == dte][['Strike','Expiry','Last','Bid','Mid']]
    for idx, row in curr_puts.iterrows():
        long_idx = idx
        for idx_j, row_j in curr_puts[curr_puts.Strike > row.Strike].iterrows():
            
            if row.Strike > 150:
                break
            if row_j.Strike - row.Strike >= 20:
                break
            
            short_idx = idx_j
            
            try:
                entry_pos = curr_puts.iloc[[long_idx,short_idx]]
                entry_pos['Pos'] = [2, -1]

                exit_pos = curr_sim.iloc[[long_idx, short_idx]][['Strike','Expiry','Simulated Prices']]
                exit_pos['Pos'] = [2, -1]
                print(entry_pos)
            
                cost.append(sum(entry_pos.Bid*entry_pos.Pos))
                new_price.append(sum(exit_pos['Simulated Prices']*exit_pos.Pos))
                
                long_strike.append(row.Strike)
                long_expiry.append(row.Expiry)
                short_strike.append(row_j.Strike)
                short_expiry.append(row_j.Expiry)

            except:
                continue
    


C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



   Strike     Expiry  Last   Bid    Mid  Pos
0    45.0 2018-09-28  0.20  0.15  0.175    2
1    50.0 2018-09-28  0.26  0.25  0.300   -1
   Strike     Expiry  Last   Bid    Mid  Pos
0    45.0 2018-09-28  0.20  0.15  0.175    2
2    52.5 2018-09-28  0.45  0.05  0.275   -1
   Strike     Expiry  Last   Bid    Mid  Pos
0    45.0 2018-09-28   0.2  0.15  0.175    2
3    53.0 2018-09-28   0.5  0.25  0.375   -1
   Strike     Expiry  Last   Bid    Mid  Pos
0    45.0 2018-09-28  0.20  0.15  0.175    2
4    54.0 2018-09-28  0.25  0.30  0.375   -1
   Strike     Expiry  Last   Bid    Mid  Pos
0    45.0 2018-09-28   0.2  0.15  0.175    2
5    55.0 2018-09-28   0.1  0.20  0.375   -1
   Strike     Expiry  Last   Bid    Mid  Pos
0    45.0 2018-09-28   0.2  0.15  0.175    2
6    56.0 2018-09-28   0.6  0.05  0.325   -1
   Strike     Expiry  Last   Bid    Mid  Pos
0    45.0 2018-09-28  0.20  0.15  0.175    2
7    56.5 2018-09-28  1.05  0.00  0.475   -1
   Strike     Expiry  Last   Bid    Mid  Pos
0    45.0 

    Strike     Expiry  Last   Bid    Mid  Pos
2     52.5 2018-09-28  0.45  0.05  0.275    2
24    68.0 2018-09-28  1.28  0.90  1.225   -1
    Strike     Expiry  Last   Bid    Mid  Pos
2     52.5 2018-09-28  0.45  0.05  0.275    2
25    69.0 2018-09-28  1.15  0.90  1.150   -1
    Strike     Expiry  Last   Bid    Mid  Pos
2     52.5 2018-09-28  0.45  0.05  0.275    2
26    69.5 2018-09-28  0.65  0.50  1.425   -1
    Strike     Expiry  Last   Bid    Mid  Pos
2     52.5 2018-09-28  0.45  0.05  0.275    2
27    70.0 2018-09-28  1.45  1.00  1.275   -1
    Strike     Expiry  Last   Bid    Mid  Pos
2     52.5 2018-09-28  0.45  0.05  0.275    2
28    71.0 2018-09-28  2.00  0.90  1.425   -1
    Strike     Expiry  Last   Bid    Mid  Pos
2     52.5 2018-09-28  0.45  0.05  0.275    2
29    71.5 2018-09-28  1.05  0.50  1.625   -1
    Strike     Expiry  Last   Bid    Mid  Pos
2     52.5 2018-09-28  0.45  0.05  0.275    2
30    72.0 2018-09-28  1.20  0.90  1.275   -1
   Strike     Expiry  Last   Bid  

19    65.5 2018-09-28  0.27  0.25  1.300   -1
    Strike     Expiry  Last  Bid    Mid  Pos
5     55.0 2018-09-28   0.1  0.2  0.375    2
20    66.0 2018-09-28   0.9  0.3  0.725   -1
    Strike     Expiry  Last   Bid    Mid  Pos
5     55.0 2018-09-28  0.10  0.20  0.375    2
21    66.5 2018-09-28  1.15  0.35  0.900   -1
    Strike     Expiry  Last  Bid    Mid  Pos
5     55.0 2018-09-28  0.10  0.2  0.375    2
22    67.0 2018-09-28  0.95  0.6  0.900   -1
    Strike     Expiry  Last   Bid    Mid  Pos
5     55.0 2018-09-28   0.1  0.20  0.375    2
23    67.5 2018-09-28   1.0  0.35  1.450   -1
    Strike     Expiry  Last  Bid    Mid  Pos
5     55.0 2018-09-28  0.10  0.2  0.375    2
24    68.0 2018-09-28  1.28  0.9  1.225   -1
    Strike     Expiry  Last  Bid    Mid  Pos
5     55.0 2018-09-28  0.10  0.2  0.375    2
25    69.0 2018-09-28  1.15  0.9  1.150   -1
    Strike     Expiry  Last  Bid    Mid  Pos
5     55.0 2018-09-28  0.10  0.2  0.375    2
26    69.5 2018-09-28  0.65  0.5  1.425   -1
   

    Strike     Expiry  Last   Bid    Mid  Pos
8     57.0 2018-09-28  0.65  0.15  0.475    2
11    59.5 2018-09-28  0.50  0.15  0.675   -1
    Strike     Expiry  Last   Bid    Mid  Pos
8     57.0 2018-09-28  0.65  0.15  0.475    2
12    60.0 2018-09-28  0.65  0.65  0.700   -1
    Strike     Expiry  Last   Bid    Mid  Pos
8     57.0 2018-09-28  0.65  0.15  0.475    2
13    62.0 2018-09-28  1.10  0.30  0.625   -1
    Strike     Expiry  Last   Bid    Mid  Pos
8     57.0 2018-09-28  0.65  0.15  0.475    2
14    62.5 2018-09-28  0.76  0.10  1.225   -1
    Strike     Expiry  Last   Bid    Mid  Pos
8     57.0 2018-09-28  0.65  0.15  0.475    2
15    63.0 2018-09-28  0.80  0.30  0.700   -1
    Strike     Expiry  Last   Bid    Mid  Pos
8     57.0 2018-09-28  0.65  0.15  0.475    2
16    63.5 2018-09-28  1.30  0.35  0.825   -1
    Strike     Expiry  Last   Bid    Mid  Pos
8     57.0 2018-09-28  0.65  0.15  0.475    2
17    64.0 2018-09-28  0.50  0.30  0.700   -1
    Strike     Expiry  Last   Bid 

...

100   136.0 2018-09-28  18.00  14.9  16.45   -1
     Strike     Expiry   Last   Bid    Mid  Pos
90    125.0 2018-09-28  12.25  11.6  12.25    2
101   137.0 2018-09-28  16.10  15.5  16.95   -1
     Strike     Expiry   Last   Bid    Mid  Pos
90    125.0 2018-09-28  12.25  11.6  12.25    2
102   138.0 2018-09-28  21.00  16.3  17.35   -1
     Strike     Expiry   Last   Bid    Mid  Pos
90    125.0 2018-09-28  12.25  11.6  12.25    2
103   139.0 2018-09-28  21.39  16.5  17.45   -1
     Strike     Expiry   Last   Bid    Mid  Pos
90    125.0 2018-09-28  12.25  11.6  12.25    2
104   140.0 2018-09-28  18.20  16.8  17.60   -1
     Strike     Expiry   Last   Bid    Mid  Pos
90    125.0 2018-09-28  12.25  11.6  12.25    2
105   141.0 2018-09-28  19.00  17.1  18.85   -1
     Strike     Expiry   Last   Bid    Mid  Pos
90    125.0 2018-09-28  12.25  11.6  12.25    2
106   142.0 2018-09-28  24.32  17.7  19.25   -1
     Strike     Expiry   Last   Bid    Mid  Pos
90    125.0 2018-09-28  12.25  11.6  12.

     Strike     Expiry   Last   Bid    Mid  Pos
94    129.0 2018-09-28  19.52  12.1  13.25    2
107   143.0 2018-09-28  16.70  18.1  19.50   -1
     Strike     Expiry   Last   Bid    Mid  Pos
94    129.0 2018-09-28  19.52  12.1  13.25    2
108   144.0 2018-09-28  19.00  18.6  20.40   -1
     Strike     Expiry   Last   Bid    Mid  Pos
94    129.0 2018-09-28  19.52  12.1  13.25    2
109   145.0 2018-09-28  20.70  19.0  20.10   -1
     Strike     Expiry   Last   Bid    Mid  Pos
94    129.0 2018-09-28  19.52  12.1  13.25    2
110   146.0 2018-09-28  22.00  20.1  21.80   -1
     Strike     Expiry   Last   Bid    Mid  Pos
94    129.0 2018-09-28  19.52  12.1  13.25    2
111   147.0 2018-09-28  22.09  19.7  21.60   -1
     Strike     Expiry   Last   Bid    Mid  Pos
94    129.0 2018-09-28  19.52  12.1  13.25    2
112   148.0 2018-09-28   9.90  20.8  22.35   -1
    Strike     Expiry   Last   Bid    Mid  Pos
95   130.0 2018-09-28  13.90  12.8  13.40    2
96   132.0 2018-09-28  14.42  13.4  14.75 

     Strike     Expiry  Last   Bid    Mid  Pos
99    135.0 2018-09-28  15.0  14.5  15.90    2
102   138.0 2018-09-28  21.0  16.3  17.35   -1
     Strike     Expiry   Last   Bid    Mid  Pos
99    135.0 2018-09-28  15.00  14.5  15.90    2
103   139.0 2018-09-28  21.39  16.5  17.45   -1
     Strike     Expiry  Last   Bid   Mid  Pos
99    135.0 2018-09-28  15.0  14.5  15.9    2
104   140.0 2018-09-28  18.2  16.8  17.6   -1
     Strike     Expiry  Last   Bid    Mid  Pos
99    135.0 2018-09-28  15.0  14.5  15.90    2
105   141.0 2018-09-28  19.0  17.1  18.85   -1
     Strike     Expiry   Last   Bid    Mid  Pos
99    135.0 2018-09-28  15.00  14.5  15.90    2
106   142.0 2018-09-28  24.32  17.7  19.25   -1
     Strike     Expiry  Last   Bid   Mid  Pos
99    135.0 2018-09-28  15.0  14.5  15.9    2
107   143.0 2018-09-28  16.7  18.1  19.5   -1
     Strike     Expiry  Last   Bid   Mid  Pos
99    135.0 2018-09-28  15.0  14.5  15.9    2
108   144.0 2018-09-28  19.0  18.6  20.4   -1
     Strike     

     Strike     Expiry   Last   Bid   Mid  Pos
104   140.0 2018-09-28  18.20  16.8  17.6    2
113   149.0 2018-09-28  19.39  21.5  23.0   -1
     Strike     Expiry   Last   Bid    Mid  Pos
104   140.0 2018-09-28  18.20  16.8  17.60    2
114   150.0 2018-09-28  24.08  22.3  23.65   -1
     Strike     Expiry  Last   Bid   Mid  Pos
104   140.0 2018-09-28  18.2  16.8  17.6    2
115   152.5 2018-09-28  25.6  23.0  24.7   -1
     Strike     Expiry  Last   Bid   Mid  Pos
104   140.0 2018-09-28  18.2  16.8  17.6    2
116   155.0 2018-09-28  26.6  24.6  26.1   -1
     Strike     Expiry   Last   Bid    Mid  Pos
105   141.0 2018-09-28  19.00  17.1  18.85    2
106   142.0 2018-09-28  24.32  17.7  19.25   -1
     Strike     Expiry  Last   Bid    Mid  Pos
105   141.0 2018-09-28  19.0  17.1  18.85    2
107   143.0 2018-09-28  16.7  18.1  19.50   -1
     Strike     Expiry  Last   Bid    Mid  Pos
105   141.0 2018-09-28  19.0  17.1  18.85    2
108   144.0 2018-09-28  19.0  18.6  20.40   -1
     Strike  

115   152.5 2018-09-28  25.60  23.0  24.70   -1
     Strike     Expiry   Last   Bid    Mid  Pos
114   150.0 2018-09-28  24.08  22.3  23.65    2
116   155.0 2018-09-28  26.60  24.6  26.10   -1
     Strike     Expiry   Last   Bid    Mid  Pos
114   150.0 2018-09-28  24.08  22.3  23.65    2
117   160.0 2018-09-28  28.70  27.0  28.55   -1
     Strike     Expiry   Last   Bid    Mid  Pos
114   150.0 2018-09-28  24.08  22.3  23.65    2
118   165.0 2018-09-28  33.18  29.9  31.75   -1


In [4]:
pnl_df = pd.DataFrame({'long_strike': long_strike,
                       'short_strike': short_strike,
                       'long_expiry': long_expiry,
                       'short_expiry':short_expiry,
                       'cost': cost,
                       'new_price': new_price},
                      index = range(len(long_strike)))

In [5]:
mu = yahoo_query('MU',dt.datetime(2018,1,1))
mu.full_info_query()

In [29]:
earnings = mu.incomeStatementQuarter['netIncomeApplicableToCommonShares'].head(3).sum() + 4325*10**6
shares_outstanding = 1159765000
currentPrice = 44.52
mkt_cap = currentPrice*shares_outstanding

In [30]:
mkt_cap/earnings

3.6528289918641668

In [31]:
(earnings/shares_outstanding)/currentPrice

0.273760420273511